In [ ]:
import numpy as np
import cv2
import matplotlib as plt
import time, csv, os, shutil, sys

# Data preparation

j=0
label=0
trainPath = "../Mood_Detector/training.csv"
dstPath = "../Mood_Detector/"
imgPath = "../Mood_Detector/Manually_Annotated/Manually_Annotated_Images/"

with open(trainPath, 'r') as trainFile:
    sp = []
    skip_to_line = 0
    for lines in trainFile:
        skip_to_line += 1
        # skip header
        if skip_to_line == 1:
            for lines in trainFile:
                sp = lines.split(',')
                # get the label
                label = sp[6]
                # move the image file path into corresponding target directory
                shutil.move(imgPath+sp[0],dstPath + label + "/")

print("done")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import matplotlib as plt
import time, csv, os, shutil, sys
from IPython.display import display, clear_output


trainPath = "../Mood_Detector/training.csv"
dstPath = "../Mood_Detector/"
imgPath = "../Mood_Detector/Manually_Annotated/Manually_Annotated_Images/"

CATEGORIES = ["Neutral","Happiness","Sadness","Surprise","Fear","Disgust","Anger","Contempt"]
training_data = []
IMG_SIZE = 64
i=0

# data preparation and converting BGR images to RGB. 
def createTrainData():    
    for category in CATEGORIES:
        path = os.path.join(dstPath, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img))
                img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
                new_array = cv2.resize(img_array,(IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass
    

createTrainData()
print("done")

In [ ]:
import random 

# shuffle the data
random.shuffle(training_data)

X = []
y = []

for features, labels in training_data:
    X.append(features)
    y.append(labels)


In [ ]:
import pickle

# we pickle the data for safe keeping 
pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [ ]:
import pickle
import numpy as np

# we Load the data set and we reshape the data.
IMG_SIZE = 64

X = pickle.load(open("../GPA_759/Data_Set/X.pickle","rb"))
y = pickle.load(open("../GPA_759/Data_Set/y.pickle","rb"))

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)

In [ ]:
print(X.shape)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)

# I convert and normalize half the data, because my RAM can't take it all...
# batch_num = 1
batch_num = 2

def getHalfData(data, batch_num):
    if batch_num == 1:
        d = data[:int(len(data)/2)]
    elif batch_num == 2 :
        d = data[int(len(data)/2)+1:]

    return d

# i'm Normalizing half the data
def dataNormalization(data, batch_num):
    if batch_num == 1:
        d = data[:int(len(data)/2),:,:,:].astype('float32')
        d /= 255
    elif batch_num == 2 :
        d = data[int(len(data)/2)+1:,:,:,:].astype('float32')
        d /= 255

    return d
    
x_train = dataNormalization(x_train, batch_num)
x_test = dataNormalization(x_test, batch_num)
y_train = getHalfData(y_train, batch_num)
y_test = getHalfData(y_test, batch_num)


print("done")

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(len(y_train))
print(len(y_test))
# print(y_train)
# print(y_test)

In [ ]:
import os
import keras
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Activation, MaxPooling2D, BatchNormalization, GlobalMaxPooling2D, Dropout, Flatten
from keras.optimizers import Adam, Adadelta
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, TensorBoard

# num_classes = 10
num_classes = 8
num_epochs = 40
num_batch = 32

# # train data and test data for cifar10 dataset
# (x_train, y_train), (x_test, y_test) = cifar10.load_data()
# print(x_train.shape)

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
# input : 64, 64 color RGB
# conv1
model.add(Conv2D(66, (3,3), padding='same', input_shape=x_train.shape[1:])) # change input shape
model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001, scale=True))
model.add(Activation('relu'))
model.add(Dropout(0.2))

# Conv 2, 3 and 4
model.add(Conv2D(128,(3,3),padding='same'))
model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001, scale=True))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Conv2D(128,(3,3)))
model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001, scale=True))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Conv2D(128,(3,3)))
model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001, scale=True))
model.add(Activation('relu'))
model.add(Dropout(0.2))

# Conv 5
model.add(Conv2D(192, (3,3)))
model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001, scale=True))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Activation('relu'))
model.add(Dropout(0.2))

# Conv 6, 7, 8 and 9
model.add(Conv2D(192,(3,3),padding='same'))
model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001, scale=True))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Conv2D(192,(3,3),padding='same'))
model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001, scale=True))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Conv2D(192,(3,3),padding='same'))
model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001, scale=True))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Conv2D(192,(3,3),padding='same'))
model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001, scale=True))
model.add(Activation('relu'))
model.add(Dropout(0.2))

# Conv 9
model.add(Conv2D(288, (3,3), padding='same'))
model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001, scale=True))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size=(2,2)))

# Conv 10
model.add(Conv2D(288, (3,3), padding='same'))
model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001, scale=True))
model.add(Activation('relu'))
model.add(Dropout(0.2))

# Conv 11
model.add(Conv2D(355, (3,3)))
model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001, scale=True))
model.add(Activation('relu'))
model.add(Dropout(0.2))

# Conv 12
model.add(Conv2D(432, (3,3)))
model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001, scale=True))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(GlobalMaxPooling2D())

# fully connected Layer MLP 
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax', bias_initializer='constant'))


# Initialise the optimizer Adam 
#optim = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
optim = Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)


# Compile the model 
model.compile(loss = 'categorical_crossentropy', optimizer=optim, metrics = ['accuracy'])

# If we are using data augmentation then we augment the images
#horizontal/vertical flip and random rotation
datagen = ImageDataGenerator(
    rotation_range=20,
    horizontal_flip=True,
    vertical_flip=True)

# apply the transformation on the train images
datagen.fit(x_train)

# we want to reduce the LR once we hit a plateau 
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001, verbose=1, mode='auto')


# Fit the model with the dataset and start training
model.fit(x_train, y_train, batch_size=num_batch, epochs=num_epochs, callbacks=[reduce_lr], validation_split=0.2, shuffle=True )

# Then we evaluate the model with evaluate()
# and we print the models score
score = model.evaluate(x_test, y_test, batch_size=num_batch)
print(score)



In [ ]:
model.save("MoodSimpNetV2-1DenseLayer-Softmax40.h5")

In [10]:
import face_recognition
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model

nameList = []
CATEGORIES = ["Neutral","Happiness","Sadness","Surprise","Fear","Disgust","Anger","Contempt"]

IMG_SIZE = 64
unl = 1

if unl == 1:
    # Load the pre-trained model only once
    model = load_model('../GPA_759/Pretrained_network/MoodSimpNetV2-1DenseLayer-Softmax40-v2.h5')

# Get a reference to webcam #0
video_capture = cv2.VideoCapture(0)


while (video_capture.isOpened()):
    
    #get a single frame of video
    ret, frame = video_capture.read()
    
    if ret:

        # resizing the frame for faster face recognition processing
        faceFrame  = cv2.resize(frame, (0,0), fx=0.25, fy=0.25)

        # we take the face location(s) in the image 
        face_locations = face_recognition.face_locations(frame)

        # Notes: instead of saving the images we could 
        # pass the image straight to the CNN for classification

        for face_location in face_locations:

            # carré 129 par 129
            top, right, bottom, left = face_location
            cropped_roi = frame[top-70:bottom+20,left-50:right+50]
            new_array = cv2.resize(cropped_roi,(IMG_SIZE, IMG_SIZE))
            new_array = cv2.cvtColor(new_array, cv2.COLOR_BGR2RGB)
            new_array = new_array[np.newaxis,:,:,:]
            new_array = new_array.astype('float32')
            new_array /= 255
            
            
            # we then classify the image and print the result
            moodPrediction = model.predict(new_array)

            # we draw rectangles around the faces
            cv2.rectangle(frame,(left-50,top-70), (right+50, bottom+20), (0,0,255), 2)
            # we draw the labels with the names over, or below the face
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, CATEGORIES[np.argmax(moodPrediction)], (left -50, bottom ), font, 1.0, (255, 255, 255), 1)

        # we Display the image
        cv2.imshow('Video',frame)

        # Hit 'q' on the keyboard to quit!
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
    else:
        break





# Release handle to the webcam
unl = 0
video_capture.release()
cv2.destroyAllWindows()



In [ ]:
import os
import keras
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Activation, MaxPooling2D, BatchNormalization, GlobalMaxPooling2D, Dropout, Flatten
from keras.optimizers import Adam, Adadelta
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, TensorBoard
from keras.models import load_model


# Here we load the model to be able to train it with another set of images.

model = load_model('../GPA_759/Pretrained_network/MoodSimpNetV2-1DenseLayer-Softmax40-v2.h5')

# num_classes = 10
num_classes = 8
num_epochs = 40
num_batch = 32

# # train data and test data for cifar10 dataset
# (x_train, y_train), (x_test, y_test) = cifar10.load_data()
# print(x_train.shape)

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


# Initialise the optimizer Adam 
#optim = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
optim = Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)


# Compile the model 
model.compile(loss = 'categorical_crossentropy', optimizer=optim, metrics = ['accuracy'])

# If we are using data augmentation then we augment the images
#horizontal/vertical flip and random rotation
datagen = ImageDataGenerator(
    rotation_range=20,
    horizontal_flip=True,
    vertical_flip=True)

# apply the transformation on the train images
datagen.fit(x_train)

# we want to reduce the LR once we hit a plateau 
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001, verbose=1, mode='auto')


# Fit the model 
model.fit(x_train, y_train, batch_size=num_batch, epochs=num_epochs, callbacks=[reduce_lr], validation_split=0.2, shuffle=True )

# Then we evaluate the model with evaluate()
# and we print the model
score = model.evaluate(x_test, y_test, batch_size=num_batch)
print(score)


In [ ]:
model.save("../GPA_759/Pretrained_network/MoodSimpNetV2-1DenseLayer-Softmax40-v2.h5")